In [ ]:
import pandas as pd

No todos los csv tienen los campos separados por `,`. En caso de que requiera utilizar otro separador, hay que indicarselo con `sep`

In [ ]:
pd.read_csv("/content/compras_caba_2016-2021.csv",sep=";")

Por defecto, pandas utiliza el encoding utf8 para abrir los archivos. En algunos casos, dependiendo del origen, tenemos que utilizar un encodig distinto porque hay caracteres que sino no va a poder interpretar

In [ ]:
distribucion = pd.read_csv("distribucion-insumos-salud-sexual-y-reproductiva.csv",sep=";",encoding = 'ISO-8859-1')

Si bien los datos aplicando alguna traduccion en los nombres podria unirlo al dataset anterior, se ve que hay alguna incongruencia entre los datos, y por tanto no tendria sentido analizarlo

In [ ]:
distribucion

En este caso, si aplico stack directamente para pasar a una representacion por fila (y asi evitar tener una matriz dispersa), la columna de insumos va a pasar tambien al indice por fila, haciendonos perder la relacion insumo-año. Primero entonces, tengo que mover la columna al indice

In [ ]:
stacked = distribucion.set_index("INSUMOS").stack()

Despues de aplicar stack, el dataframe pasa a ser una serie al perder el indice por columna. Aun asi, puedo acceder a los datos por el indice por fila utilizando loc

In [ ]:
stacked.loc["IMPLANTES"]

In [ ]:
df_stacked = stacked.to_frame()

In [ ]:
df_stacked.info()

In [ ]:
distribucion.info()

Parece poco intuitivo, pero el uso de memoria es mayor en la matriz dispersa (que tiene que tener celdas con nans) que la representacion mas compacta. Esto se debe a que el indice ahora ocupa mucho mas espacio al tener que mantener los elementos y la tabla de traduccion. 

Podemos ver que si reseteamos los indices y transformamos las columnas a categorias, el uso de memoria es igual a la version con multiindice. 

In [ ]:
reset = df_stacked.reset_index()

In [ ]:
reset.INSUMOS = reset.INSUMOS.astype("category")
reset.level_1 = reset.level_1.astype("category")
reset

In [ ]:
reset.info()

Si revisamos una de las columnas que son categoricas, vemos que pandas nos muestra no solo los datos, sino la tabla de traduccion que utiliza para poder comprimir los datos originales. Notar que es una lista de python, entonces los datos los puede comprimir a enteros, y el tamaño del entero (int8/16/32/64) va a depender de la cantidad de datos unicos; haciendo que a mayor cantidad no solo aumente el uso de memoria por la tabla de traduccion sino porque tiene que utilizar un tipo de dato mas grande para reprensentar los elementos

In [ ]:
reset.INSUMOS